<a href="https://colab.research.google.com/github/AnetaGancheva/Notebooks/blob/main/MultiAgentSystem_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Calculate the distance to the furthest Irish pub in the world

A Multi Agent System practice:

Current scope: System of agents will calculate the travel distance between current position and the furthest Irish pub in the world.

In [2]:
!pip install 'smolagents[litellm]' matplotlib geopandas shapely kaleido -q

In [3]:
import math
from typing import Tuple
from smolagents import tool

@tool
def calculate_distance(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
) -> float:

    """
    A function to calculate the distance between two points on Earth based on their coordinates.

    Args:

    origin_coords: Tuple of (latitude, longitude) for starting point
    destination_coords: Tuple of (latitude, longitude) for end point

    Returns:
    float: the calculated distance in km between the two points

    Example:
        >>> # Varna, Bulgaria (43.2141° N, 27.9147° E) to Santiago, Chille (33.4489° S, 70.6693° W)
        >>> result = calculate_distance((43.2141, 27.9147), (-33.4489, -70.6693))
    """

    # Convert degrees to radians function
    def to_rads(degrees: float) -> float:
      return (degrees*(math.pi/180.0))

    # Extract coordinates
    lat_orig, lon_orig = map(to_rads, origin_coords)
    lat_dest, lon_dest = map(to_rads, destination_coords)

    # Earth's radius (km)
    EARTH_RADIUS_KM = 6371.0

    # Apply Haversine formula to get distance
    dlat = lat_dest - lat_orig
    dlon = lon_dest - lon_orig

    a = (
        math.sin(dlat/2) ** 2 + math.cos(lat_orig) * math.cos(lat_dest) * math.sin(dlon/2) ** 2
    )

    c = 2 * math.asin(math.sqrt(a))

    distance = EARTH_RADIUS_KM * c

    return distance


In [4]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel, VisitWebpageTool

model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

In [5]:
web_agent = CodeAgent(
    model= model,
    tools=[ DuckDuckGoSearchTool(), VisitWebpageTool(), calculate_distance ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

In [6]:
import os
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel
from PIL import Image


def check_reasoning_and_plot(final_answer, agent_memory):
    final_answer
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True

In [9]:
manager_agent = CodeAgent(
    model=HfApiModel("deepseek-ai/DeepSeek-R1", provider="together"),
    tools=[calculate_distance],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=1
)

In [ ]:
manager_agent.run("""
Find the locations of all Irish pubs in the world and calculate the distance to Varna, Bulgaria (43.2141° N, 27.9147° E).
Represent this as a spatial map of the world, with the locations represented as scatter ploints with a color that depends on travel time.

Here is an example of how to plot and return a map:

df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you will see it.

""")